<a href="https://colab.research.google.com/github/keqiu1898/final-project/blob/main/websiteapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Backgroud**

A biopsy with a thorough pathological examination is the gold standard for determining if a suspicious mole is cancerous. However, biopsies are not always needed or appropriate due to the associated costs or preexisting conditions that make healing more difficult. It is especially true for sensitive areas like the face or ears. Biopsies can also cause undue stress and anxiety for patients and their loved ones. 

Due to these concerns, we are turning to non-invasive techniques that can give us helpful information instead of – or before – a traditional biopsy. We built a model that automatically classifies skin images into eleven categories: five benign skin lesions - nevus; keratosis including seborrheic keratosis, pigmented benign keratosis, lichenoid keratosis; lentigo including solar lentigo, lentigo NOS, and lentigo simplex; dermatofibroma and all the other benign skin lesion like scars, angioma, neurofibroma, verruca, acrochordon, etc. included in the category named other benign skin lesions. Three skin cancers: melanoma, basal cell carcinoma, and squamous cell carcinoma. Three indeterminate skin lesions are called actinic keratosis, vascular lesion, and all other indeterminate skin lesions. 
Our model requests users to upload a clear skin lesion image to make our classification more accurate. The training data are from ISIC. It is challenging owing to the fine-grained variability in the appearance of skin lesions. Luckily, our model works well using a search tree combined with convolutional neural networks. 

## **Libraries**

In [ ]:
!pip install gradio

In [3]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
import numpy as np

from PIL import Image
import requests
from io import BytesIO

Mounted at /content/gdrive/


## **Structure**

**Step 1**

**invalid - 0 ; valid - 1**

**Step 2**

**determinate - 0 ; indeterminate - 1**

**step 3**

**benign - 0 ; malignant - 1**

**step4-1**

**keratosis - 0, lentigo - 1, nevus - 2, dermatofibroma - 3, neurofibroma - 4**



**step4-2**

**melanoma - 0, basal_cell_carcinoma - 1, squamous_cell_carcinoma - 2**



**step 4-3**

**actinic keratosis - 0, vascular lesion - 1, and others - 2**.

## **Model Uploading**

In [5]:
#classes = ['invalid', 'valid']

firststep = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/Valid_Invalid_ResNet')
secondstep = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/Deter_Indeter_ResNet')
thirdstep = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/BM_AlexNet')
fourth_one = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/benign_DenseNet')
fourth_two = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/malignant_VGG')
fourth_three = keras.models.load_model('/content/gdrive/MyDrive/ColabNotebooks/lastversionskincancer/indeterminate_ResNet101V2')


## **Searching Tree**

In [9]:

def predict_lesion(img):
  #img_3d = np.array(img, dtype = 'uint8')/255
  # im_resize = img_3d.reshape(-1, 250, 250, 3)
  im_resize = np.array([img],  dtype='uint8')/255

  # step 1
  pred_invalid = np.argmax(firststep.predict(im_resize, verbose = 1), axis=1)
  if pred_invalid[0] == 0: # an invalid skin image
      out = 'Please Upload a Clear Image of Skin Lesion.'
      return out
  pred_deter = np.argmax(secondstep.predict(im_resize, verbose = 1), axis=1) 
  if pred_deter[0] == 1:  # an indeterminate skin lesion
     pred_indeter_class = np.argmax(fourth_three.predict(im_resize, verbose = 1), axis=1)
     out = ' it is an INDETERMINATE skin lesion; please consult a dermatologist further. '
     if pred_indeter_class[0] == 0:
        return ' most likely Actinic Keratosis; ' + out
     elif pred_indeter_class[0] == 1:
        return  ' most likely Vascular Lesion; ' +out
     else:
       return ' hard to give a specific lesion name; ' + out
  pred_BM = np.argmax(thirdstep.predict(im_resize, verbose = 1), axis=1) 
  if pred_BM[0] == 0: # it is a benign skin lesion
     pred_benign_class = np.argmax(fourth_one.predict(im_resize, verbose = 1), axis=1)
     out = 'It is a BENIGN skin lesion; '
     if pred_benign_class[0] == 0:
        return out + ' most likely (benign) Keratosis.'
     elif pred_benign_class[0] == 1:
        return out + ' most likely Lentigo.'
     elif pred_benign_class[0] == 2:
        return out + ' most likely Nevus.'
     elif pred_benign_class[0] == 3:
        return out + ' most likely Dermatofibroma.'        
     else:
       return out + 'but hard to give a specific lesion name.'

  pred_malignant_class = np.argmax(fourth_two.predict(im_resize, verbose = 1), axis=1)
  out = 'It is SKIN CANCER ! '
  if pred_malignant_class[0] == 0:
      return out + ' most likely Melanoma!'
  elif pred_malignant_class[0] == 1:
      return out + ' most likely Basal Cell Carcinoma!'
  return out + ' most likely Squamous Cell Carcinoma!' 



## **Application**

In [ ]:
image = gr.inputs.Image(shape = (250,250))
label = gr.outputs.Label(num_top_classes = 1)
gr.Interface(fn=predict_lesion, inputs= image, outputs=label, capture_session = True).launch(debug = True, share= True)